In [11]:
# This main block of generator script which will be used to create stego image at sender end
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

# Load the trained generator model
generator = tf.keras.models.load_model('/Users/anichougule/Masters/Code/Python/SEM3/Computer Vision/cs512-f24-aniket-chougule/Project/Data/generator_model_nlp.h5')

# Function to convert text to a binary string
def text_to_binary_string(text):
    return ''.join(format(ord(char), '08b') for char in text)

# Function to map binary data to a noise vector
def map_binary_to_noise(binary_string, target_dim=100, sigma=3):
    noise_vector = []
    for i in range(0, len(binary_string), sigma):
        bit_group = binary_string[i:i + sigma]
        if len(bit_group) < sigma:
            bit_group = bit_group.ljust(sigma, '0')  # Pad with zeros if needed
        m = int(bit_group, 2)
        noise_value = (m / (2**sigma - 1)) * 2 - 1  # Scale to [-1, 1]
        noise_vector.append(noise_value)
    if len(noise_vector) < target_dim:
        repeat_count = (target_dim - len(noise_vector)) // len(noise_vector)
        remainder = (target_dim - len(noise_vector)) % len(noise_vector)
        noise_vector = np.concatenate([noise_vector] * (repeat_count + 2))[:target_dim]
    else:
        noise_vector = np.array(noise_vector[:target_dim])
    return noise_vector

# Generate and save a stego image
def generate_stego_image(text, output_dir='/Users/anichougule/Downloads/', image_name='stego_image.png'):
    os.makedirs(output_dir, exist_ok=True)
    
    # Convert the text to a binary string and map to noise
    binary_string = text_to_binary_string(text)
    print(binary_string)
    noise_vector = map_binary_to_noise(binary_string, target_dim=100)
    noise_vector = np.expand_dims(noise_vector, axis=0)  # Prepare for model input
    
    # Generate the stego image
    stego_image = generator.predict(noise_vector)
    
    # Rescale image to [0, 255] and save
    image = (stego_image[0] * 127.5 + 127.5).astype(np.uint8)
    plt.imsave(os.path.join(output_dir, image_name), image)

# Example usage
test_sentence = "This the testing of Stegnography using GAN. "
generate_stego_image(test_sentence, image_name='test_stego_image.png')


0110001001101001011011100110000101110010011110010101111101110011011101000111001001101001011011100110011100100000001111010010000001110100011001010111100001110100010111110111010001101111010111110110001001101001011011100110000101110010011110010101111101110011011101000111001001101001011011100110011100101000011101000110010101111000011101000010100100001010011100000111001001101001011011100111010000101000011000100110100101101110011000010111001001111001010111110111001101110100011100100110100101101110011001110010100100001010011011100110111101101001011100110110010101011111011101100110010101100011011101000110111101110010001000000011110100100000011011010110000101110000010111110110001001101001011011100110000101110010011110010101111101110100011011110101111101101110011011110110100101110011011001010010100001100010011010010110111001100001011100100111100101011111011100110111010001110010011010010110111001100111001011000010000001110100011000010111001001100111011001010111010001011111011001000110100101101101